In [91]:
import re # Regular Expressions

In [92]:
f = open("datos.xls","r") # Se abre el fichero original corrupto [r: read(estamos abriendo un archivo para leerlo)]
a = f.read() 
print(a) # Esto es todo lo que tiene el xml

<?xml version="1.0" encoding="ISO-8859-1"?>
<Workbook xmlns="urn:schemas-microsoft-com:office:spreadsheet" xmlns:user="urn:my-scripts" xmlns:msxsl="urn:schemas-microsoft-com:xslt" xmlns:fo="http://www.w3.org/1999/XSL/Format" xmlns:html="http://www.w3.org/TR/REC-html40" xmlns:ss="urn:schemas-microsoft-com:office:spreadsheet" xmlns:x="urn:schemas-microsoft-com:office:excel" xmlns:o="urn:schemas-microsoft-com:office:office"><ExcelWorkbook xmlns="urn:schemas-microsoft-com:office:excel"><WindowHeight>9345</WindowHeight><WindowWidth>15180</WindowWidth><WindowTopX>120</WindowTopX><WindowTopY>60</WindowTopY><ProtectStructure>False</ProtectStructure><ProtectWindows>False</ProtectWindows></ExcelWorkbook><ss:Styles><ss:Style ss:Name="normal-centrado" ss:ID="normal-centrado"><NumberFormat ss:Format="@"/><Alignment ss:WrapText="0" ss:VerticalText="0" ss:Vertical="Center" ss:ShrinkToFit="0" ss:Rotate="0.0" ss:ReadingOrder="LeftToRight" ss:Indent="0" ss:Horizontal="Center"/><ss:Interior ss:Color="Aut

In [93]:
# findall : Returns a list containing all matches
datos = re.findall("<Data(.+?)>(.+?)</Data>", a) 
#datos = re.findall("<Data([^>]+?)>([^<]+?)</Data>", a) # En las celdas en blanco del excel, no me devuelve nada
# [0-9]{0,4}
lista = []

for i in datos:
    print(i[1])
    lista.append(i[1])

Control de Grupos de Matrícula
Año académico:
2019/20-0
Fecha de creación:
25/02/2019
Centro (desde):
13 - Escuela Politécnica Superior
Centro (hasta):
13 - Escuela Politécnica Superior
Plan (desde):
264 - MÁSTER UNIVERSITARIO EN INGENIERÍA INFORMÁTICA (PLAN 2018)
Plan (hasta):
264 - MÁSTER UNIVERSITARIO EN INGENIERÍA INFORMÁTICA (PLAN 2018)
Tipología académica:
Todas
Plan 264 - MÁSTER UNIVERSITARIO EN INGENIERÍA INFORMÁTICA (PLAN 2018)
Asignatura
Alumnos
Criterio asignación
Profesor
Código
Descripción
Curso
Grupo
Tipología académica
Activ.
Tp
Vp
Turno
Máximos propios
Matriculados
Propios
Externos
Progr. inter.
Total
Código
Descripción
Asig. alfb
PF
NIU
Nombre y Apellidos
CDS
Actas
Docencia
Responsable
Rep.
No Rep.
Rep.
No Rep.
Máx.
Exce.
Asig.
Disp.
Máx.
Asig.
Disp.
desde
hasta
8111
GESTIÓN TÉCNICA Y ESTRATÉGICA TIC
1
90
Teoría
S
S
1
Mixto
0
15
0
0
15
0
0
15
0
0
0
0
0
1
Manual
</Cell><Cell ss:MergeDown="0" ss:MergeAcross="0" ss:StyleID="normal-cent-linea-negr"><Data ss:Type="String"/>

In [94]:
#print(datos) #Lista de tuplas
print(len(lista)) # Lista 

457


In [95]:
filas=0
for n, i in enumerate(lista):
    if len(i) > 100:
        filas +=1
        #print(i)
        #ista[n] = 10
print(filas)

14


In [96]:
# Función para contar el número de veces que 
# aparece una letra en una cadena de caracteres
def contarLetra(string, letra):
    contador = 0
    #string = string.upper()
    #letra = letra.upper()
    for elemento in string:
        if elemento == letra:
            contador += 1
    return contador

In [108]:
lista_buena = [] #Lista donde se almacenarán todos los datos del excel de forma correcta.
num = 0

for n, i in enumerate(lista):    # Se recorre la lista con etiquetas xml no deseadas.
    if len(i) > 120: # Si la longitud de la cadena es superior, significa que hay etiquetas xml no deseadas y hay que modificar la lista. 
        #print(i)
        #print("\n")
        num = contarLetra(i,"I") # Se podría contar los "Data" en vez de las "I", pero es más sencillo contar un caracter.
        #print(num)

        while( num > 0 ): # Si num es 2, significa que hay 2 celdas en blanco, luego hay que meter 2 posiciones en blanco.
            lista_buena.append("") # Se añade una posición en blanco
            num -= 1 
            
        lista_buena.append(i[-4:])# Añadimos el número de 4 dígitos que se encontraba al final de la etiqueta xml no deseada, para que no se pierda información.
        
    else: # No hay que modificar la lista, luego añadimos la información tal cual. 
        lista_buena.append(i)

print(len(lista_buena))   #Tiene que dar 520 celdas

520


In [98]:
for n, i in enumerate(lista_buena):
    print(lista_buena[n])

Control de Grupos de Matrícula
Año académico:
2019/20-0
Fecha de creación:
25/02/2019
Centro (desde):
13 - Escuela Politécnica Superior
Centro (hasta):
13 - Escuela Politécnica Superior
Plan (desde):
264 - MÁSTER UNIVERSITARIO EN INGENIERÍA INFORMÁTICA (PLAN 2018)
Plan (hasta):
264 - MÁSTER UNIVERSITARIO EN INGENIERÍA INFORMÁTICA (PLAN 2018)
Tipología académica:
Todas
Plan 264 - MÁSTER UNIVERSITARIO EN INGENIERÍA INFORMÁTICA (PLAN 2018)
Asignatura
Alumnos
Criterio asignación
Profesor
Código
Descripción
Curso
Grupo
Tipología académica
Activ.
Tp
Vp
Turno
Máximos propios
Matriculados
Propios
Externos
Progr. inter.
Total
Código
Descripción
Asig. alfb
PF
NIU
Nombre y Apellidos
CDS
Actas
Docencia
Responsable
Rep.
No Rep.
Rep.
No Rep.
Máx.
Exce.
Asig.
Disp.
Máx.
Asig.
Disp.
desde
hasta
8111
GESTIÓN TÉCNICA Y ESTRATÉGICA TIC
1
90
Teoría
S
S
1
Mixto
0
15
0
0
15
0
0
15
0
0
0
0
0
1
Manual


1015739
PRG SC PRAS AN SEIS MAS SEIS, SANTIAGO ARCOS ARCOS
S
N
S
N
8112
ARQUITECTURAS AVANZADAS DE COMPUTAD

In [71]:
f.close()

In [39]:
#Ejemplo de búsqueda
xx = "guru99, education is fun"
r1 = re.findall(r"^\w+",xx)
print(r1)

['guru99']


In [103]:
mystr = "abcdefgh1234" 
print(mystr[-4:]) 


1234
